# Virtual Canva

In [2]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import Button, Frame, filedialog, colorchooser
from PIL import Image, ImageTk, ImageGrab  # Added ImageGrab import
import time

### Calibrating target object

In [2]:
# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Intializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 tracbars that will control the lower and upper range of H,S & V channels.
# The Arguments are like this: Name of trackbar, window name, range, callback function.
# For Hue the range is 0-179 and for S,V its 0-255.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
 
while True:
    
    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    if not ret:
        break
    # Flip the frame horizontally (Not required)
    frame = cv2.flip( frame, 1 ) 
    
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time as the user changes them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # Set the lower and upper HSV range according to the value selected by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask, where white represents your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # You can also visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Converting the binary mask to 3 channel image, this is just so we can stack it with the others
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack the mask, orginal frame and the filtered result
    stacked = np.hstack((mask_3,frame,res))
    
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    # If the user presses `s` then print this array.
    if key == ord('s'):
        
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as penval.npy
        np.save('penval',thearray)
        break
    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

### Virtual Pen and Eraser on a canvas

In [3]:
# Load calibration file
load_from_disk = True
if load_from_disk:
    penval = np.load("penval.npy")

# Video capture
cap = cv2.VideoCapture(0)

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Background subtractor
backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
background_threshold = 600
switch = 'Pointer'
last_switch = time.time()

# Initialize points
x1, y1 = None, None  # Initialize x1, y1 as None
noiseth = 800
wiper_thresh = 40000
clear = False
pen_color = 'blue'  # Default pen color

# Tkinter setup
root = tk.Tk()
root.title("Finger Drawing App")

# Frame for buttons
button_frame = Frame(root)
button_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

# Canvas for drawing
canvas = tk.Canvas(root, width=800, height=600, bg='white')
canvas.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)

# Button functions
def set_tool(tool):
    global switch
    canvas.delete("pointer")  # Remove pointer indicator when switching tools
    switch = tool

def clear_canvas():
    canvas.delete("all")

def change_color():
    global pen_color
    color = colorchooser.askcolor(title="Choose a color")[1]
    if color:
        pen_color = color

def save_drawing():
    # Save the current canvas content as a PNG file directly
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
    if file_path:
        x = root.winfo_rootx() + canvas.winfo_x()
        y = root.winfo_rooty() + canvas.winfo_y()
        x1 = x + canvas.winfo_width()
        y1 = y + canvas.winfo_height()
        ImageGrab.grab().crop((x, y, x1, y1)).save(file_path, 'PNG')

def set_background_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", ".png;.jpg;*.jpeg")])
    if file_path:
        bg_image = Image.open(file_path)
        bg_image = bg_image.resize((800, 600), Image.Resampling.LANCZOS)  # Updated attribute
        bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.background = bg_photo  # Keep a reference.
        canvas.create_image(0, 0, image=bg_photo, anchor='nw')

# Buttons
btn_pointer = Button(button_frame, text="Pointer", command=lambda: set_tool("Pointer"))
btn_pen = Button(button_frame, text="Pen", command=lambda: set_tool("Pen"))
btn_eraser = Button(button_frame, text="Eraser", command=lambda: set_tool("Eraser"))
btn_clear = Button(button_frame, text="Clear Canvas", command=clear_canvas)
btn_color = Button(button_frame, text="Change Color", command=change_color)
btn_save = Button(button_frame, text="Save Drawing", command=save_drawing)
btn_background = Button(button_frame, text="Set Background", command=set_background_image)
btn_pointer.pack(pady=5)
btn_pen.pack(pady=5)
btn_eraser.pack(pady=5)
btn_clear.pack(pady=5)
btn_color.pack(pady=5)
btn_save.pack(pady=5)
btn_background.pack(pady=5)

# Label for displaying frames
frame_label = tk.Label(root)
frame_label.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)

def update_frame():
    global x1, y1, clear, last_switch, pen_color
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    top_left = frame[0: 50, 0: 50]
    fgmask = backgroundobject.apply(top_left)
    switch_thresh = np.sum(fgmask == 255)
    
    if switch_thresh > background_threshold and (time.time() - last_switch) > 1:
        last_switch = time.time()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > noiseth:
        c = max(contours, key=cv2.contourArea)
        x2, y2, w, h = cv2.boundingRect(c)
        
        if switch == 'Pointer':
            canvas.delete("pointer")
            canvas.create_oval(x2-10, y2-10, x2+10, y2+10, outline="green", width=2, tags="pointer")
        elif switch == 'Pen':
            canvas.delete("pointer")
            if x1 is not None and y1 is not None:
                canvas.create_line(x1, y1, x2, y2, fill=pen_color, width=5)
        elif switch == 'Eraser':
            canvas.delete("pointer")
            canvas.create_oval(x2-20, y2-20, x2+20, y2+20, fill="white", outline="white", width=20)
        
        x1, y1 = x2, y2
        
        if cv2.contourArea(c) > wiper_thresh:
            clear = True
    else:
        x1, y1 = None, None

    if clear:
        canvas.delete("all")
        clear = False

    frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_photo = ImageTk.PhotoImage(image=frame_image)
    frame_label.imgtk = frame_photo
    frame_label.configure(image=frame_photo)
    frame_label.after(10, update_frame)

update_frame()
root.mainloop()

cap.release()
cv2.destroyAllWindows()
